In [1]:
!pip install rdflib
!pip install opendatasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import FOAF, SKOS, RDF, RDFS, XSD, OWL
import opendatasets as od
import pandas as pd

## Download the datasets from kaggle

In [3]:
od.download('https://www.kaggle.com/datasets/nobelfoundation/nobel-laureates')
# TODO download the other ones

Skipping, found downloaded files in "./nobel-laureates" (use force=True to force download)


## Read the csv file

In [4]:
nobels = pd.read_csv('nobel-laureates/archive.csv')
nobels

,Year,Category,Prize,Motivation,Prize Share,Laureate ID,Laureate Type,Full Name,Birth Date,Birth City,Birth Country,Sex,Organization Name,Organization City,Organization Country,Death Date,Death City,Death Country
0,1901,Chemistry,The Nobel Prize in Chemistry 1901,"""in recognition of the extraordinary services ...",1/1,160,Individual,Jacobus Henricus van 't Hoff,1852-08-30,Rotterdam,Netherlands,Male,Berlin University,Berlin,Germany,1911-03-01,Berlin,Germany
1,1901,Literature,The Nobel Prize in Literature 1901,"""in special recognition of his poetic composit...",1/1,569,Individual,Sully Prudhomme,1839-03-16,Paris,France,Male,NaN,NaN,NaN,1907-09-07,Châtenay,France
2,1901,Medicine,The Nobel Prize in Physiology or Medicine 1901,"""for his work on serum therapy, especially its...",1/1,293,Individual,Emil Adolf von Behring,1854-03-15,Hansdorf (Lawice),Prussia (Poland),Male,Marburg University,Marburg,Germany,1917-03-31,Marburg,Germany
3,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,462,Individual,Jean Henry Dunant,1828-05-08,Geneva,Switzerland,Male,NaN,NaN,NaN,1910-10-30,Heiden,Switzerland
4,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,463,Individual,Frédéric Passy,1822-05-20,Paris,France,Male,NaN,NaN,NaN,1912-06-12,Paris,France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,2016,Medicine,The Nobel Prize in Physiology or Medicine 2016,"""for his discoveries of mechanisms for autophagy""",1/1,927,Individual,Yoshinori Ohsumi,1945-02-09,Fukuoka,Japan,Male,Tokyo Institute of Technology,Tokyo,Japan,NaN,NaN,NaN
965,2016,Peace,The Nobel Peace Prize 2016,"""for his resolute efforts to bring the country...",1/1,934,Individual,Juan Manuel Santos,1951-08-10,Bogotá,Colombia,Male,NaN,NaN,NaN,NaN,NaN,NaN
966,2016,Physics,The Nobel Prize in Physics 2016,"""for theoretical discoveries of topological ph...",1/2,928,Individual,David J. Thouless,1934-09-21,Bearsden,United Kingdom,Male,University of Washington,"Seattle, WA",United States of America,NaN,NaN,NaN
967,2016,Physics,The Nobel Prize in Physics 2016,"""for theoretical discoveries of topological ph...",1/4,929,Individual,F. Duncan M. Haldane,1951-09-14,London,United Kingdom,Male,Princeton University,"Princeton, NJ",United States of America,NaN,NaN,NaN


## Parse our ontology

In [5]:
graph = Graph()
graph.parse('nobelOntology.ttl', format='turtle')
graph.parse('http://eulersharp.sourceforge.net/2003/03swap/countries', format='turtle')

NOBEL = Namespace('http://www.semanticweb.org/a3d/ontologies/2024/10/nobelOntology/')
JUR = Namespace('http://sweet.jpl.nasa.gov/2.3/humanJurisdiction.owl#')

# binding prefixes to URIs
graph.bind('nobel', NOBEL)
graph.bind('jur', JUR)

for ns_prefix, namespace in graph.namespaces():
    print('{}: {}'.format(ns_prefix, namespace))

for s, p, o in graph.triples((None, RDF.type, JUR.Country)):
    print(f"{s} is a country")

brick: https://brickschema.org/schema/Brick#
csvw: http://www.w3.org/ns/csvw#
dcat: http://www.w3.org/ns/dcat#
dcmitype: http://purl.org/dc/dcmitype/
dcam: http://purl.org/dc/dcam/
doap: http://usefulinc.com/ns/doap#
foaf: http://xmlns.com/foaf/0.1/
geo: http://www.opengis.net/ont/geosparql#
odrl: http://www.w3.org/ns/odrl/2/
org: http://www.w3.org/ns/org#
prof: http://www.w3.org/ns/dx/prof/
prov: http://www.w3.org/ns/prov#
qb: http://purl.org/linked-data/cube#
schema: https://schema.org/
sh: http://www.w3.org/ns/shacl#
skos: http://www.w3.org/2004/02/skos/core#
sosa: http://www.w3.org/ns/sosa/
ssn: http://www.w3.org/ns/ssn/
time: http://www.w3.org/2006/time#
vann: http://purl.org/vocab/vann/
void: http://rdfs.org/ns/void#
wgs: https://www.w3.org/2003/01/geo/wgs84_pos#
owl: http://www.w3.org/2002/07/owl#
rdf: http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs: http://www.w3.org/2000/01/rdf-schema#
xsd: http://www.w3.org/2001/XMLSchema#
xml: http://www.w3.org/XML/1998/namespace
jur: http:

## Populate the graph with nobel-laureates

In [6]:
import datetime, unicodedata, html, re

def normalize_name(raw_name):
    name = html.unescape(raw_name)
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('ascii')
    uri_name = name.replace(',', '')
    uri_name = uri_name.replace('-', ' ')
    uri_name = ''.join(x for x in uri_name.title() if not x.isspace())
    return (name, uri_name)

def handle_city(raw_city):
    (city_name, uri_city_name) = normalize_name(raw_city)
    city = URIRef(NOBEL[uri_city_name])
    if (city, RDF.type, NOBEL.City) not in graph: # new city
        graph.add((city, RDF.type, NOBEL.City))
        graph.add((city, FOAF.name, Literal(city_name, datatype=XSD.string)))
    return city

def handle_org(raw_org):
    (org_name, uri_org_name) = normalize_name(raw_org)
    org = URIRef(NOBEL[uri_org_name])
    if (org, RDF.type, FOAF.Organization) not in graph: # new organization
        graph.add((org, RDF.type, FOAF.Organization))
        graph.add((org, FOAF.name, Literal(org_name, datatype=XSD.string)))
    return org

def handle_city_country(country_name):
    # special cases

    if country_name == 'United States of America':
        country_name = 'United States'
    if country_name == 'Scotland' or country_name == 'Northern Ireland':
        country_name = 'United Kingdom'
    if country_name == 'Guadeloupe Island':
        country_name = 'Guadeloupe'
    if country_name == 'East Timor':
        country_name = 'Timor-Leste'
    if country_name == 'East Germany' or country_name == 'Federal Republic of Germany':
        country_name = 'Germany'
    if country_name == 'Union of Soviet Socialist Republics':
        country_name = 'Russian Federation'
    if country_name == 'Czechoslovakia':
        country_name = 'Czech Republic'
    if country_name == 'Vietnam':
        country_name = 'Viet Nam'
    
    # Regex pattern to capture content inside parentheses
    pattern = r"\((.*?)\)"
    match = re.findall(pattern, country_name)
    if len(match) != 0:
        country_name = match[0]

    if country_name == 'Republic of Macedonia':
        country_name = 'Macedonia, the former Yugoslav Republic of'
    if country_name == 'South Korea':
        country_name = 'Korea, Republic of'
    if country_name == "People's Republic of China":
        country_name = 'China'
    if country_name == 'then Germany, now France':
        country_name = 'France'
    
    country_query = f'''
    SELECT ?country
    WHERE {{
        ?country rdf:type jur:Country;
                foaf:name ?name.
        FILTER(REGEX(?name, "{country_name}"))
    }}'''

    qres = graph.query(country_query)
    if (len(qres) == 0):
        print('Country not found: {}'.format(country_name))
        return None

    return qres


for index, row in nobels.iterrows():
    nobel = URIRef(NOBEL[row['Category'] + str(row['Year'])]) # the URI will be nobelNamespace + Category + Year
    graph.add((nobel, RDF.type, NOBEL.NobelPrize))
    graph.add((nobel, NOBEL.hasYear, Literal(row['Year'], datatype=XSD.gYear)))
    graph.add((nobel, NOBEL.hasNobelCategory, Literal(row['Category'], datatype=XSD.string)))

    # handle Prize Share
    prizeShare = str(row['Prize Share']).split('/')
    graph.add((nobel, NOBEL.hasPrizeShare, Literal(prizeShare[1], datatype=XSD.integer)))

    # handle Motivation
    if (pd.notna(row['Motivation'])):
        # TODO topic analysis
        graph.add((nobel, NOBEL.hasMotivationTopics, Literal(row['Motivation'], datatype=XSD.string)))
    
    # handle Laureate Type
    if row['Laureate Type'] == 'Organization':
        (org_name, uri_org_name) = normalize_name(str(row['Full Name']))
        laureate = URIRef(NOBEL[uri_org_name])
        graph.add((laureate, RDF.type, FOAF.Organization))
        graph.add((laureate, FOAF.name, Literal(org_name, datatype=XSD.string)))
    elif row['Laureate Type'] == 'Individual':
        laureate = URIRef(NOBEL[str(row['Laureate ID'])]) # the URI will be nobelNamespace + Laureate ID
        graph.add((laureate, RDF.type, FOAF.Person))
        graph.add((laureate, FOAF.name, Literal(row['Full Name'], datatype=XSD.string)))
        graph.add((laureate, FOAF.gender, Literal(row['Sex'], datatype=XSD.string)))

    graph.add((laureate, RDF.type, NOBEL.Laureate))
    
    if pd.notna(row['Birth Date']):
        try:
            datetime.datetime.strptime(str(row['Birth Date']), '%Y-%m-%d')
            graph.add((laureate, NOBEL.birthDate, Literal(row['Birth Date'], datatype=XSD.date)))
        except ValueError:
            splitted_date = str(row['Birth Date']).split('-')
            new_date = splitted_date[0] + '-01-01'
            print('Wrong Birth Date format in {}. The new date will be {}'.format(laureate, new_date))
            graph.add((laureate, NOBEL.birthDate, Literal(new_date, datatype=XSD.date)))

    if pd.notna(row['Death Date']):
        try:
            datetime.datetime.strptime(str(row['Death Date']), '%Y-%m-%d')
            graph.add((laureate, NOBEL.deathDate, Literal(row['Death Date'], datatype=XSD.date)))
        except ValueError:
            splitted_date = str(row['Death Date']).split('-')
            new_date = splitted_date[0] + '-01-01'
            print('Wrong Death Date format in {}. The new date will be {}'.format(laureate, new_date))
            graph.add((laureate, NOBEL.deathDate, Literal(new_date, datatype=XSD.date)))

    # handle birth city
    if pd.notna(row['Birth City']):
        birth_city = handle_city(str(row['Birth City']))
        graph.add((laureate, NOBEL.bornIn, birth_city))

    # handle birth city country
    if pd.notna(row['Birth Country']):
        qres = handle_city_country(str(row['Birth Country']))
        if (qres is not None) and ((birth_city, NOBEL.locatedIn, JUR.Country) not in graph): # new locatedIn
            graph.add((birth_city, NOBEL.locatedIn, next(iter(qres)).country)) # only the first match

    # handle death city
    if pd.notna(row['Death City']):
        death_city = handle_city(str(row['Death City']))
        graph.add((laureate, NOBEL.diedIn, death_city))

    # handle death city country
    if pd.notna(row['Death Country']):
        qres = handle_city_country(str(row['Death Country']))
        if (qres is not None) and ((death_city, NOBEL.locatedIn, JUR.Country) not in graph): # new locatedIn
            graph.add((death_city, NOBEL.locatedIn, next(iter(qres)).country)) # only the first match

    # handle organization
    if pd.notna(row['Organization Name']):
        org = handle_org(str(row['Organization Name']))
        graph.add((laureate, NOBEL.worksFor, org))

        if pd.notna(row['Organization City']):
            org_city = handle_city(str(row['Organization City']))
            graph.add((org, NOBEL.basedIn, org_city))

        if pd.notna(row['Organization Country']):
            qres = handle_city_country(str(row['Organization Country']))
            if (qres is not None) and ((org_city, NOBEL.locatedIn, JUR.Country) not in graph): # new locatedIn
                graph.add((org_city, NOBEL.locatedIn, next(iter(qres)).country)) # only the first match

Wrong Birth Date format in http://www.semanticweb.org/a3d/ontologies/2024/10/nobelOntology/519. The new date will be 1898-01-01
Wrong Birth Date format in http://www.semanticweb.org/a3d/ontologies/2024/10/nobelOntology/745. The new date will be 1943-01-01


## Serialization

In [7]:
with open('test.ttl', 'w') as out:
    out.write(graph.serialize(format='turtle'))